In [ ]:
from typing import List, Dict
from marketplace.app import MarketPlaceApp
import os

# Setting the client_id of the app and pass the access_token to using the app through SDK
mp = MarketPlaceApp(client_id="5fd66c68-50e9-474a-b55d-148777ae3efd",
                    access_token=os.environ.get('MP_ACCESS_TOKEN'))

# The heartbeat capability to show the state of server and the HPC resource
print(mp.heartbeat())

## initialize a new job

using capability `new_trasformation`, the app create a folder under the user repository path in the remote HPC resource. 
The remote path name is returned for futher files transfor operations and as `transformation_id` for job submission.

In [ ]:
# The resourceId returned will be the remote path
response = mp.new_transformation(config=None)
print(response)

import ast
response = ast.literal_eval(response)
remote_path = response['resourceId']

In [ ]:
!ssh daint.cscs.ch "cd /scratch/snx3000/jyu/firecrest/jusong_yu/ && ls -l"

# Upload input files and slurm job script to the path

The capabilities `updateDataset`, `getDataset` and `deleteDataset` are for upload, download and delete files on remote path.

However, instead of remote path only it is also require to pass filename to manipulate the files which is not completely compatible with the initial designs of these capabilities which are for database in the first place. 
I hack it a bit to pass the filename by a json object.

In [ ]:
cmd = f'ls -l /scratch/snx3000/jyu/firecrest/jusong_yu/{remote_path}'
!ssh daint.cscs.ch $cmd

In [ ]:
! head -n 10 ./sample_files/colloid.in

In [ ]:
# send a file to remote folder
with open('./sample_files/colloid.in', 'rb') as fh:
    mp.put(
        path='updateDataset',
        params={'resource': f'{remote_path}'},
        files={'file': fh},
    )
    
with open('./sample_files/submit.sh', 'rb') as fh:
    mp.put(
        path='updateDataset',
        params={'resource': f'{remote_path}'},
        files={'file': fh},
    )

In [ ]:
cmd = f'ls -l /scratch/snx3000/jyu/firecrest/jusong_yu/{remote_path}'
!ssh daint.cscs.ch $cmd

In [ ]:
# should be `mp.get_dataset` in the future
resp = mp.get(
    path='getDataset', 
    params={'resource': f'{remote_path}'}, 
    json={"filename": "submit.sh"},
)

In [ ]:
print(resp.text)

## Submit job and check the jobs list

`start_transformation` corresponding to capability `startTransformation` which will do actuall job submit into the scheduler system and return the job info.
Then I can using `get_transformation_list` to retrieve all the jobs of current user in remote resource.

TODO: implement `get_transformation_status`

In [ ]:
mp.get_transformation_list()

In [ ]:
mp.post(
    path='startTransformation',
    params={'resource': f'{remote_path}'}
).text

In [ ]:
mp.get_transformation_list()

In [ ]:
cmd = f"tail /scratch/snx3000/jyu/firecrest/jusong_yu/{remote_path}/log.lammps"
!ssh daint.cscs.ch $cmd

## Notes to activate account for using hpc-app

1. Having an account on MarketPlace.
2. Send me the email and I will add it to whitelist of app.
3. https://mp-hpc.herokuapp.com/register with MarketPlace token which will create data repo (a remote folder) in HPC resource and mapping the user name to hpc server database where the jobs and users are storing. Currently it is in `/scratch/snx3000/jyu/firecrest/<prefer_name>/`. (TODO: this will be a view form in frontpage.)
4. Happy computing.